In [64]:
import datetime
import email
import imaplib
import mailbox
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import glob
from numpy import nansum
from numpy import nanmean
import datetime
import functools
from datetime import datetime

In [65]:
import math
from scipy.optimize import brentq
Gamma=0.66

#----------------numerical solution--------------------------
# Tw + (es*(Tw))/𝛾 = Ta + ea/ 𝛾
# Ta air temperature 
# RH relative humidit
def TwNumerical(Ta,RH):
    SaturatedVPTa = 6.11* np.exp( (17.269*Ta) / (237.3+Ta) )
    VaporPressure = SaturatedVPTa*RH/100.0    
    def GetRoot(Tw):
        SaturatedVPTw = 6.11* np.exp( (17.269*Tw) / (237.3+Tw) )
        Function = Tw + SaturatedVPTw/Gamma-Ta-VaporPressure/Gamma
        return Function
    root,info = brentq(GetRoot,-20,50, full_output=True)
    return root


#----------------stull's formula----------------------
# use Stull's formula to calculate wet bulb temperature
# Ta air temperature 
# RH relative humidity
def Tw_Stull(Ta, RH):
    Tw_Stull=Ta*np.arctan(0.151977*(RH+8.313659)**0.5)+\
            np.arctan(Ta+RH)-np.arctan(RH-1.676331)+\
            np.arctan(0.023101*RH)*0.00391838*RH**(3./2.)-4.686035
    return Tw_Stull

def Clean_table_TW(df, st, tr):

    df['Lon'] = df['Lon'].str.replace('W','-').str.replace('E','').str.replace(' ','').astype(float)
    df['Lat'] = df['Lat'].str.replace('S','-').str.replace('N','').str.replace(' ','').astype(float)
    df['Temp'] = df['Temp'].str.replace('C','').str.replace(' ','').astype(float)
    df['Temp']=(df['Temp']-32)*5/9
    df['RH'] = df['RH'].str.replace('%','').str.replace(' ','').astype(float)
    df['Time']= pd.to_datetime(st[4:6]+'-'+st[-2:]+'-'+st[0:4]+ ' ' + df['Time'])
    #df['Biked']=np.nan
    #df['Biked'][0]=(df['Time'][df.shape[0]-1]-df['Time'][0]).seconds/60
    #df['Date']=np.nan
    #df['Date'][0]=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    #df['Date_all']=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    #df['Time']=df['Time'].replace(year=int(st[0:4]), month=int(st[4:6]), day=int(st[-2:]))
    df['Tw_Numerical']=df.apply(lambda x: TwNumerical(x['Temp'], x['RH']), axis=1)
    df['Tw_Stull']=Tw_Stull(df['Temp'], df['RH'])
    
    
    
    
    df = df.fillna('')
    return df

In [66]:
Dat_list=glob.glob('Bike study-6/*.txt')
Dat_list

['Bike study-6\\20191002-evening.txt',
 'Bike study-6\\20191002-morning.txt',
 'Bike study-6\\20191003-evening.txt',
 'Bike study-6\\20191003-morning.txt',
 'Bike study-6\\20191004-evening.txt',
 'Bike study-6\\20191004-morning.txt',
 'Bike study-6\\20191022-evening.txt',
 'Bike study-6\\20191022-morning.txt',
 'Bike study-6\\20191023-evening.txt',
 'Bike study-6\\20191023-morning.txt',
 'Bike study-6\\20191025-evening.txt',
 'Bike study-6\\20191025-morning.txt',
 'Bike study-6\\20191028-evening.txt',
 'Bike study-6\\20191028-morning.txt',
 'Bike study-6\\20191029-evening.txt',
 'Bike study-6\\20191029-morning.txt',
 'Bike study-6\\20191030-evening.txt',
 'Bike study-6\\20191030-morning.txt',
 'Bike study-6\\20191031-evening.txt',
 'Bike study-6\\20191031-morning .txt',
 'Bike study-6\\20191115.txt',
 'Bike study-6\\20191118.txt',
 'Bike study-6\\20191120-morning.txt',
 'Bike study-6\\20191120.txt',
 'Bike study-6\\20191122.txt',
 'Bike study-6\\20191125.txt',
 'Bike study-6\\20200224e

In [67]:
# Read csvs and clean data
Dat_all=[]
ii=1
for item in Dat_list:
    Dat=pd.read_csv(item, names=["Time", "Lon", "Lat", "ID","Temp","RH"]) 
    Dat_clean=Clean_table_TW(Dat, str(item)[13:21], "{:0>2}".format(str(ii)))
    Dat_all.append(Dat_clean)
    ii=ii+1


In [68]:
Dat_all

[                    Time       Lon       Lat     ID       Temp     RH  \
 0    2019-10-02 08:08:12 -72.90217  41.35806   AFF9  23.500000  79.62   
 1    2019-10-02 08:08:13 -72.90217  41.35806   AFF9  23.527778  79.54   
 2    2019-10-02 08:08:14 -72.90216  41.35806   AFF9  23.527778  79.46   
 3    2019-10-02 08:08:15 -72.90216  41.35806   AFF9  23.500000  79.38   
 4    2019-10-02 08:08:16 -72.90217  41.35805   AFF9  23.527778  79.34   
 ...                  ...       ...       ...    ...        ...    ...   
 3806 2019-10-02 17:41:11 -72.90223  41.35813   AFF9  27.238889  64.31   
 3807 2019-10-02 17:41:12 -72.90223  41.35813   AFF9  27.222222  64.29   
 3808 2019-10-02 17:41:13 -72.90223  41.35813   AFF9  27.222222  64.32   
 3809 2019-10-02 17:41:14 -72.90221  41.35813   AFF9  27.238889  64.27   
 3810 2019-10-02 17:41:15 -72.90221  41.35813   AFF9  27.238889  64.21   
 
       Tw_Numerical   Tw_Stull  
 0        20.923797  20.838650  
 1        20.939260  20.854587  
 2        2

In [69]:
# Dat_all_trimmed=[]
# for item in Dat_all:
#     Temp=item.iloc[240:]
#     Temp=Temp[:-240]
#     Dat_all_trimmed.append(Temp)

In [70]:
#Dat_all_trimmed

In [71]:
#Concatanate into one data frame
result= pd.concat(Dat_all, ignore_index=True)
result
##Remove duplicate datetimes
result=result.drop_duplicates(subset=['Time']).sort_values(['Time']).reset_index()
result

,index,Time,Lon,Lat,ID,Temp,RH,Tw_Numerical,Tw_Stull
0,0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,20.923797,20.838650
1,1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,20.939260,20.854587
2,2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,20.928626,20.843884
3,3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,20.891918,20.806557
4,4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,20.912667,20.827826
...,...,...,...,...,...,...,...,...,...
112548,143666,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,5.803058,5.305589
112549,143667,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,5.766360,5.268015
112550,143668,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,5.757764,5.259496
112551,143669,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,5.747017,5.248692


In [72]:
## Add Biked and Date column
result['Date']=np.nan
result['Date_all']=np.nan
result['Biked']=np.nan
kk=0
for index, row in result.iterrows():
    result['Date_all'][index]=str(result['Time'][kk])  
    if (index<=len(result)-2):
        if(((result['Time'][index+1]-result['Time'][index]).seconds>=300)):
            result['Biked'][kk]=(result['Time'][index]-result['Time'][kk]).seconds/60
            result['Date'][kk]=str(result['Time'][kk])
            result['Date_all'][index]=str(result['Time'][kk])
            kk=index+1
    else:
        result['Biked'][kk]=(result['Time'][index]-result['Time'][kk]).seconds/60
        result['Date'][kk]=str(result['Time'][kk])
        result['Date_all'][index]=str(result['Time'][kk])
        kk=index+1
        #result['Biked'][index]=np.nan
        #result['Date'][index]=np.nan
#     if (index==0):
#         #result['Biked'][index]=np.nan
#         #result['Date'][index]=np.nan
#         result['Date_all'][index]=str(result['Time'][index])
#         result['Date'][index]=str(result['Time'][index])
#         result['Biked'][index]=(result['Time'][kk]-result['Time'][index]).seconds/60

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\tirth\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while w

In [86]:
#Fin=result[(result['Biked']<=240) | (result['Biked']).isnull()]
#Remove cases where someone biked for 0 time
Fin=result[(result['Biked']>=5) | (result['Biked']).isnull()]
Fin=Fin[['Time', 'Lon', 'Lat', 'ID', 'Temp', 'RH', 'Biked','Date','Date_all','Tw_Numerical','Tw_Stull']]
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826
...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692


In [87]:
Fin['Temp_norm'] = Fin.groupby('Date_all')['Temp'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['Temp_norm'] = Fin['Temp']-Fin['Temp_norm']
Fin['RH_norm'] = Fin.groupby('Date_all')['RH'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['RH_norm'] = Fin['RH']-Fin['RH_norm']
Fin['Tw_Numerical_norm'] = Fin.groupby('Date_all')['Tw_Numerical'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['Tw_Numerical_norm'] = Fin['Tw_Numerical']-Fin['Tw_Numerical_norm']
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650,0.932870,0.664032,0.870447
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587,0.944444,0.661599,0.873753
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884,0.944444,0.659167,0.871479
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557,0.932870,0.656735,0.863631
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826,0.944444,0.655518,0.868067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589,0.099819,0.790284,0.128323
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015,0.095281,0.783550,0.117791
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496,0.096189,0.776335,0.115323
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692,0.096189,0.770563,0.112239


In [88]:
Fin['Temp_norm'][Fin['Temp_norm'] > 0.95] = np.nan
Fin['Temp_norm'][Fin['Temp_norm'] < 0.05] = np.nan
Fin['RH_norm'][Fin['RH_norm'] > 0.95] = np.nan
Fin['RH_norm'][Fin['RH_norm'] < 0.05] = np.nan
Fin['Tw_Numerical_norm'][Fin['Tw_Numerical_norm'] > 0.95] = np.nan
Fin['Tw_Numerical_norm'][Fin['Tw_Numerical_norm'] < 0.05] = np.nan

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [89]:
Fin['Temp_norm'] = Fin.groupby('Date_all')['Temp'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['Temp_norm'] = Fin['Temp']-Fin['Temp_norm']
Fin['RH_norm'] = Fin.groupby('Date_all')['RH'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['RH_norm'] = Fin['RH']-Fin['RH_norm']
Fin['Tw_Numerical_norm'] = Fin.groupby('Date_all')['Tw_Numerical'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
#Fin['Tw_Numerical_norm'] = Fin['Tw_Numerical']-Fin['Tw_Numerical_norm']
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650,0.932870,0.664032,0.870447
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587,0.944444,0.661599,0.873753
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884,0.944444,0.659167,0.871479
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557,0.932870,0.656735,0.863631
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826,0.944444,0.655518,0.868067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589,0.099819,0.790284,0.128323
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015,0.095281,0.783550,0.117791
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496,0.096189,0.776335,0.115323
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692,0.096189,0.770563,0.112239


In [90]:
Temp_ind = Fin.groupby('Date_all')['Temp_norm'].cumcount() % 9
RH_ind = Fin.groupby('Date_all')['RH_norm'].cumcount() % 9
Tw_ind = Fin.groupby('Date_all')['Tw_Numerical_norm'].cumcount() % 9
Fin['Temp_norm'][Temp_ind != 0]=np.nan
Fin['RH_norm'][Temp_ind != 0]=np.nan
Fin['Tw_Numerical_norm'][Temp_ind != 0]=np.nan
Fin

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650,0.932870,0.664032,0.870447
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587,NaN,NaN,NaN
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884,NaN,NaN,NaN
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557,NaN,NaN,NaN
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589,NaN,NaN,NaN
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015,0.095281,0.783550,0.117791
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496,NaN,NaN,NaN
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692,NaN,NaN,NaN


In [91]:
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650,0.932870,0.664032,0.870447
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587,NaN,NaN,NaN
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884,NaN,NaN,NaN
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557,NaN,NaN,NaN
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589,NaN,NaN,NaN
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015,0.095281,0.783550,0.117791
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496,NaN,NaN,NaN
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692,NaN,NaN,NaN


In [92]:
Fin.to_csv('Compiled_Sara_norm.csv')

In [93]:
ss=Fin.groupby('Date').max()
Final=pd.DataFrame(ss.index)

Final[Final['Date']==""]=np.nan
Final=Final.dropna()
Final

,Date
0,2019-10-02 08:08:12
1,2019-10-02 17:13:48
2,2019-10-03 08:11:55
3,2019-10-03 16:57:48
4,2019-10-04 08:11:33
5,2019-10-22 08:24:14
6,2019-10-22 17:12:18
7,2019-10-23 17:06:32
8,2019-10-28 17:18:29
9,2019-10-29 08:22:27


In [94]:
Final.to_csv('Compiled_Sara_Date.csv')

In [95]:
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,39.4,2019-10-02 08:08:12,2019-10-02 08:08:12,20.923797,20.838650,0.932870,0.664032,0.870447
1,2019-10-02 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,NaN,NaN,2019-10-02 08:08:12,20.939260,20.854587,NaN,NaN,NaN
2,2019-10-02 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,NaN,NaN,2019-10-02 08:08:12,20.928626,20.843884,NaN,NaN,NaN
3,2019-10-02 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,NaN,NaN,2019-10-02 08:08:12,20.891918,20.806557,NaN,NaN,NaN
4,2019-10-02 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,NaN,NaN,2019-10-02 08:08:12,20.912667,20.827826,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112548,2020-03-12 13:44:27,-72.90227,41.35798,AFF9,9.200000,60.01,NaN,NaN,2020-03-12 13:20:58,5.803058,5.305589,NaN,NaN,NaN
112549,2020-03-12 13:44:28,-72.90227,41.35798,AFF9,9.172222,59.87,NaN,NaN,2020-03-12 13:20:58,5.766360,5.268015,0.095281,0.783550,0.117791
112550,2020-03-12 13:44:30,-72.90227,41.35798,AFF9,9.177778,59.72,NaN,NaN,2020-03-12 13:20:58,5.757764,5.259496,NaN,NaN,NaN
112551,2020-03-12 13:44:33,-72.90229,41.35799,AFF9,9.177778,59.60,NaN,NaN,2020-03-12 13:20:58,5.747017,5.248692,NaN,NaN,NaN


In [97]:
Fin['Temp_norm'].min()

0.0